In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import time
import gc
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
train = pd.read_hdf('../input/train.h5')
train.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,1.0,NaN,NaN,NaN,1,NaN,104,6.0,9.0,18,1438873,2019-06-07 15:32:01,8,2.135019e+09,0,2329670524,601
1,135939,893,NaN,NaN,NaN,NaN,1,NaN,19,6.0,8.0,0,1185582,2019-06-08 19:40:40,4,2.782306e+09,1,2864801071,1000
2,399254,821,NaN,760.0,NaN,360.0,1,NaN,559,NaN,8.0,0,1555716,2019-06-06 23:59:13,0,1.392806e+09,2,628911675,696
3,68983,1004,1.0,2214.0,NaN,1080.0,0,NaN,129,2.0,8.0,0,1093419,2019-06-09 09:00:12,0,3.562553e+09,3,1283809327,753
4,288999,1076,1.0,2280.0,NaN,1080.0,1,1.0,64,2.0,8.0,0,1400089,2019-06-07 08:28:13,5,2.364522e+09,4,1510695983,582


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   android_id  500000 non-null  int32  
 1   apptype     500000 non-null  int16  
 2   carrier     435093 non-null  float64
 3   dev_height  392986 non-null  float16
 4   dev_ppi     116709 non-null  float16
 5   dev_width   392989 non-null  float16
 6   label       500000 non-null  int8   
 7   lan         316720 non-null  float64
 8   media_id    500000 non-null  int16  
 9   ntt         479383 non-null  float16
 10  osv         493439 non-null  float64
 11  package     500000 non-null  int16  
 12  sid         500000 non-null  int32  
 13  timestamp   500000 non-null  object 
 14  version     500000 non-null  int64  
 15  fea_hash    499910 non-null  float64
 16  location    500000 non-null  int16  
 17  fea1_hash   500000 non-null  int64  
 18  cus_type    500000 non-null  int16  
dtypes:

In [4]:
test1 = pd.read_hdf('../input/test2.h5')
test1.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,0,1001,1.0,640.0,NaN,360.0,1.0,29,5.0,8.0,4,1177564,2019-06-08 15:18:31,0.0,3.426912e+09,34,2694799399,411
1,0,1106,1.0,NaN,NaN,NaN,NaN,39,2.0,8.0,65,1822122,2019-06-04 21:53:18,0.0,1.331186e+09,48,4190764663,411
2,0,1001,1.0,640.0,NaN,360.0,1.0,29,2.0,6.0,4,1178302,2019-06-08 20:28:43,0.0,1.118061e+09,25,2610913319,411
3,196009,1001,1.0,2016.0,NaN,1080.0,1.0,29,6.0,7.0,4,1269787,2019-06-08 15:51:17,3.0,3.780086e+09,3,2711576615,905
4,80444,1076,1.0,2280.0,NaN,1080.0,1.0,64,2.0,9.0,0,1604520,2019-06-05 23:22:29,5.0,1.274342e+09,108,51536943,734


In [5]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   android_id  150000 non-null  int32  
 1   apptype     150000 non-null  int16  
 2   carrier     130369 non-null  float64
 3   dev_height  117568 non-null  float16
 4   dev_ppi     34910 non-null   float16
 5   dev_width   117571 non-null  float16
 6   lan         94563 non-null   float64
 7   media_id    150000 non-null  int16  
 8   ntt         143774 non-null  float16
 9   osv         148095 non-null  float64
 10  package     150000 non-null  int16  
 11  sid         150000 non-null  int32  
 12  timestamp   150000 non-null  object 
 13  version     149999 non-null  float64
 14  fea_hash    149968 non-null  float64
 15  location    150000 non-null  int16  
 16  fea1_hash   150000 non-null  int64  
 17  cus_type    150000 non-null  int16  
dtypes: float16(4), float64(5), int16(5), int32(2

In [6]:
data = pd.concat([train, test1])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 650000 entries, 0 to 149999
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   android_id  650000 non-null  int32  
 1   apptype     650000 non-null  int16  
 2   carrier     565462 non-null  float64
 3   dev_height  510554 non-null  float16
 4   dev_ppi     151619 non-null  float16
 5   dev_width   510560 non-null  float16
 6   label       500000 non-null  float64
 7   lan         411283 non-null  float64
 8   media_id    650000 non-null  int16  
 9   ntt         623157 non-null  float16
 10  osv         641534 non-null  float64
 11  package     650000 non-null  int16  
 12  sid         650000 non-null  int32  
 13  timestamp   650000 non-null  object 
 14  version     649999 non-null  float64
 15  fea_hash    649878 non-null  float64
 16  location    650000 non-null  int16  
 17  fea1_hash   650000 non-null  int64  
 18  cus_type    650000 non-null  int16  
dtypes:

发现：android_id为0的有164928个，比其他的特征要高出了100多个数量级，猜测，0可能是缺失值的一种编码方式

In [8]:
data['android_id'].value_counts()[:10]

0         165013
1190         272
6964         117
16662         39
2604          38
8183          36
44716         26
43274         26
19949         24
114020        23
Name: android_id, dtype: int64

In [9]:
cate_cols = ['android_id', 'apptype', 'carrier', 'lan', 'media_id', 'ntt', 'osv',
             'version', 'location', 'cus_type', 'package']

print('There are about {} categorical features.'.format(len(cate_cols)))

There are about 11 categorical features.


In [10]:
# count编码
for col in cate_cols: 
    print(col)
    data[col + '_cnt_code'] = data[col].map(data[col].value_counts()) 

android_id
apptype
carrier
lan
media_id
ntt
osv
version
location
cus_type
package


In [11]:
data['user_id'] = data['package'].astype(str) + '_' + data['version'].astype(str) + '_' + data['android_id'].astype(str) + '_' + data['carrier'].astype(str) + '_' + data['osv'].astype(str) + '_' + data['lan'].astype(str)

In [12]:
data['day'] = data['timestamp'].astype('datetime64').dt.day
data['hour'] = data['timestamp'].astype('datetime64').dt.hour
data['dayofweek'] = data['timestamp'].astype('datetime64').dt.dayofweek

data['time'] = (data['day'] - data['day'].min()) * 24 + data['hour']

In [13]:
time_cols = ['day', 'hour', 'dayofweek', 'time']

# count编码
for col in time_cols: 
    print(col)
    data[col + '_cnt_code'] = data[col].map(data[col].value_counts()) 

day
hour
dayofweek
time


## 用户特征1
1. 用户每天在某个app上出现的次数
2. 用户每天，每小时出现的次数
3. 用户距上次出现的时间差

In [14]:
# 用户每天/每小时在某个app上出现的次数
for i in ['day', 'time']:
    print('user_id_' + i + '_app')
    data['user_id_' + i + '_app'] = data['user_id'] + '_' + data[i].astype(str) + '_' + data['media_id'].astype(str)
    dic_ = data['user_id_' + i + '_app'].value_counts().to_dict()
    data['user_id_' + i + '_app_count'] = data['user_id_' + i + '_app'].apply(lambda x: dic_[x])
    data.drop('user_id_' + i + '_app', axis=1, inplace = True)

user_id_day_app
user_id_time_app


In [15]:
# 用户每天,每个小时出现的次数
for i in ['day', 'time']:
    print('user_id_' + i)
    data['user_id_' + i] = data['user_id'] + '_' + data[i].astype(str)
    dic_ = data['user_id_' + i].value_counts().to_dict()
    data['user_id_' + i + '_count'] = data['user_id_' + i].apply(lambda x: dic_[x])
    data.drop('user_id_' + i, axis=1, inplace=True)

user_id_day
user_id_time


In [16]:
# 用户本次出现距上次时间差
# data['user_to_lasttime'] = data.groupby('user_id')['time'].diff().values

In [17]:
for col in data.columns:
    if col not in ['sid', 'label', 'timestamp']: 
        if data[col].dtypes == 'O':
            print(col)
            data[col+'_labelencode'] = LabelEncoder().fit_transform(data[col].values)

user_id


In [18]:
# 用户每天/每小时在某个app上出现的次数
for i in ['day','time']:
    for c in ['fea_hash','fea1_hash']:
        print('user_id_'+ i +'_' +c)
        data['user_id_'+ i + '_' +c] = data['user_id']  + '_' + data[i].astype(str) + '_' + data[c].astype(str)
        dic_ = data['user_id_'+ i +'_' +c].value_counts().to_dict()
        data['user_id_'+ i +'_' + c + '_count'] = data['user_id_'+ i +'_' + c].apply(lambda x: dic_[x])
        data.drop('user_id_'+i +'_' + c, axis=1,inplace = True)

user_id_day_fea_hash
user_id_day_fea1_hash
user_id_time_fea_hash
user_id_time_fea1_hash


In [19]:
train_data = data.iloc[:train.shape[0], :]
test1_data = data.iloc[train.shape[0]:, :]

train_data_y = train_data['label']
train_data_X = train_data.drop('label', axis=1)

test1_data_y = test1_data['label']
test1_data_X = test1_data.drop('label', axis=1)

In [20]:
train_data_X_train, train_data_X_val, train_data_y_train, train_data_y_val = train_test_split(train_data_X, train_data_y, random_state=2020)

In [21]:
train_cols = [i for i in train_data_X_train.columns if i not in ['sid', 'label'] and train_data_X_train[i].dtypes != 'O']

In [22]:
params = {'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'auc',
          'num_leaves': 31,
          'learning_rate': 0.05,
          'feature_fraction': 0.95,
          'bagging_fraction': 0.85,
          'bagging_freq': 5, 
          'min_data_in_leaf':15}

In [23]:
use_cols = []
useless_cols = []

for i in train_cols:
    print(i)
    
    lgb_train = lgb.Dataset(train_data_X_train[[i]].values, train_data_y_train) 
    lgb_eval= lgb.Dataset(train_data_X_val[[i]].values, train_data_y_val, reference=lgb_train)
    lgb_test = lgb.train(params,
                         lgb_train,
                         num_boost_round=100,
                         valid_sets=[lgb_eval, lgb_train],
                         early_stopping_rounds=20,
                         verbose_eval=10)
    
    print('*' * 5)
    print(lgb_test.best_score['valid_0']['auc'])
    if lgb_test.best_score['valid_0']['auc'] > 0.52:
        use_cols.append(i)
    else:
        useless_cols.append(i)
    print('*' * 20)
    print('\n')

android_id
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.563345	valid_0's auc: 0.557344
[20]	training's auc: 0.5639	valid_0's auc: 0.557248
Early stopping, best iteration is:
[1]	training's auc: 0.561893	valid_0's auc: 0.55773
*****
0.5577299690097062
********************


apptype
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.81659	valid_0's auc: 0.816981
[20]	training's auc: 0.817316	valid_0's auc: 0.817574
[30]	training's auc: 0.817625	valid_0's auc: 0.817837
[40]	training's auc: 0.818037	valid_0's auc: 0.818302
[50]	training's auc: 0.818378	valid_0's auc: 0.818613
[60]	training's auc: 0.818515	valid_0's auc: 0.818814
[70]	training's auc: 0.818598	valid_0's auc: 0.8189
[80]	training's auc: 0.818663	valid_0's auc: 0.818977
[90]	training's auc: 0.818767	valid_0's auc: 0.819133
[100]	training's auc: 0.818811	valid_0's auc: 0.819139
Did not meet early stopping. Best iteration is:
[99]	training's auc: 0.818811	va

Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.564224	valid_0's auc: 0.563575
[20]	training's auc: 0.564225	valid_0's auc: 0.56357
[30]	training's auc: 0.564264	valid_0's auc: 0.563597
[40]	training's auc: 0.564264	valid_0's auc: 0.563598
[50]	training's auc: 0.564264	valid_0's auc: 0.563603
[60]	training's auc: 0.564274	valid_0's auc: 0.563603
[70]	training's auc: 0.564273	valid_0's auc: 0.563607
[80]	training's auc: 0.564274	valid_0's auc: 0.563608
[90]	training's auc: 0.564274	valid_0's auc: 0.563607
[100]	training's auc: 0.564274	valid_0's auc: 0.563602
Did not meet early stopping. Best iteration is:
[96]	training's auc: 0.564274	valid_0's auc: 0.563602
*****
0.5636022821825052
********************


apptype_cnt_code
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.81733	valid_0's auc: 0.817818
[20]	training's auc: 0.81778	valid_0's auc: 0.818175
[30]	training's auc: 0.818033	valid_0's auc: 0.81845
[40]	trainin

Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.524132	valid_0's auc: 0.519221
[20]	training's auc: 0.525206	valid_0's auc: 0.519701
[30]	training's auc: 0.525454	valid_0's auc: 0.520527
[40]	training's auc: 0.52583	valid_0's auc: 0.520663
[50]	training's auc: 0.526141	valid_0's auc: 0.520727
[60]	training's auc: 0.526348	valid_0's auc: 0.520665
[70]	training's auc: 0.526494	valid_0's auc: 0.520407
Early stopping, best iteration is:
[51]	training's auc: 0.526173	valid_0's auc: 0.520731
*****
0.5207311308236257
********************


user_id_day_app_count
Training until validation scores don't improve for 20 rounds
[10]	training's auc: 0.694799	valid_0's auc: 0.692353
[20]	training's auc: 0.6948	valid_0's auc: 0.692356
[30]	training's auc: 0.694814	valid_0's auc: 0.692369
[40]	training's auc: 0.694822	valid_0's auc: 0.692377
[50]	training's auc: 0.694853	valid_0's auc: 0.69241
[60]	training's auc: 0.694884	valid_0's auc: 0.692394
Early stopping, best 

In [24]:
print(use_cols)

['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi', 'dev_width', 'lan', 'media_id', 'ntt', 'osv', 'package', 'version', 'location', 'fea1_hash', 'cus_type', 'android_id_cnt_code', 'apptype_cnt_code', 'carrier_cnt_code', 'lan_cnt_code', 'media_id_cnt_code', 'ntt_cnt_code', 'osv_cnt_code', 'version_cnt_code', 'location_cnt_code', 'cus_type_cnt_code', 'package_cnt_code', 'time', 'time_cnt_code', 'user_id_day_app_count', 'user_id_time_app_count', 'user_id_day_count', 'user_id_time_count', 'user_id_labelencode', 'user_id_day_fea1_hash_count', 'user_id_time_fea1_hash_count']


In [25]:
print(useless_cols)

['fea_hash', 'day', 'hour', 'dayofweek', 'day_cnt_code', 'hour_cnt_code', 'dayofweek_cnt_code', 'user_id_day_fea_hash_count', 'user_id_time_fea_hash_count']


In [26]:
lgb_train = lgb.Dataset(train_data_X_train[use_cols], train_data_y_train)
lgb_eval = lgb.Dataset(train_data_X_val[use_cols], train_data_y_val, reference=lgb_train)

In [27]:
params = {'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'binary_logloss',
          'num_leaves': 31,
          'learning_rate': 0.05,
          'feature_fraction': 0.95,
          'bagging_fraction': 0.85,
          'bagging_freq': 5, 
          'min_data_in_leaf':15}

In [28]:
print('Start training...')

gbm_val_1 = lgb.train(params,
                      lgb_train,
                      num_boost_round=2000,
                      valid_sets=[lgb_train, lgb_eval],
                      early_stopping_rounds=50,
                      verbose_eval=200)

Start training...
Training until validation scores don't improve for 50 rounds
[200]	training's binary_logloss: 0.305808	valid_1's binary_logloss: 0.309476
[400]	training's binary_logloss: 0.295627	valid_1's binary_logloss: 0.303497
[600]	training's binary_logloss: 0.288556	valid_1's binary_logloss: 0.300705
[800]	training's binary_logloss: 0.282952	valid_1's binary_logloss: 0.299381
[1000]	training's binary_logloss: 0.277813	valid_1's binary_logloss: 0.298519
Early stopping, best iteration is:
[1132]	training's binary_logloss: 0.274673	valid_1's binary_logloss: 0.297936


In [29]:
train_data_X_val['prob'] = gbm_val_1.predict(train_data_X_val[use_cols])
train_data_X_val['pred'] = np.where(train_data_X_val['prob'] > 0.5, 1, 0)

acc = np.round(accuracy_score(train_data_y_val, train_data_X_val['pred']), 4)
auc = roc_auc_score(train_data_y_val, train_data_X_val['prob'])

print('acc: ', acc)
print('auc: ', auc)

acc:  0.8865
auc:  0.9418467391751222


In [30]:
lgb_train = lgb.Dataset(train_data_X[use_cols], train_data_y)   

print('Start training...')

gbm_1  = lgb.train(params,
                   lgb_train,
                   num_boost_round=gbm_val_1.best_iteration + 20,
                   verbose_eval=10)
print('Done')

Start training...
Done


In [31]:
test1['label'] = np.where(gbm_1.predict(test1_data_X[use_cols]) > 0.5, 1, 0)

test1[['sid', 'label']].to_csv('../sub/sub_{}_{}.csv'.format(time.strftime('%Y%m%d'), acc), index=False)